# Grey Clustering Analysis - Assessment of Natural Disaster Events in the Rio Grande do Sul Area

## 1. Disaster Data

In [35]:
import pandas as pd

In [77]:
BASE_FILE_PATH  = '../Data/Danos_Informados_{year}.xls'
AVAILABLE_YEARS = range(2013, 2024+1)
disasters_data  = pd.DataFrame()

In [78]:
for year in AVAILABLE_YEARS:
    year_data = pd.read_excel(BASE_FILE_PATH.format(year=year), skiprows=4)
    year_data['Ano'] = year

    disasters_data = pd.concat([disasters_data, year_data])


In [79]:
CATEGORIES_TO_REMOVE = ['15110 - Doenças infecciosas virais', '15120 - Doenças infecciosas bacterianas', '15210 - Infestações de animais', '15230 - Outras Infestações', '22410 - Transporte de produtos perigosos rodoviário', '23110 - Incêndios em plantas e distritos industriais, parques e depósitos', '23120 - Incêndios em aglomerados residenciais', '24100 - Colapso de edificações', '25100 - Transporte passageiros e cargas não perigosas rodoviário']
disasters_data = disasters_data.drop(disasters_data[disasters_data['COBRADE'].isin(CATEGORIES_TO_REMOVE)].index)

## 2. Clustering of Regions Regarding Natural Disaster Affects

The first thing we need to do is group the data that we have (from each municipality) into "intro-regions", as shown in the image bellow. 
![](../Docs/Regiões%20Intermediárias.png)

The data of which intro-region delimitation can be found at IBGE's web-site.

In [87]:
REGIONS_DELIMITATION = pd.read_excel('../Data/regioes_geograficas_composicao_por_municipios_2017_20180911.xls')
REGIONS_DELIMITATION = REGIONS_DELIMITATION[['nome_mun', 'nome_rgint']]
REGIONS_DELIMITATION = REGIONS_DELIMITATION[REGIONS_DELIMITATION['nome_rgint'].isin(['Porto Alegre', 'Pelotas', 'Santa Maria', 'Uruguaiana', 'Ijuí', 'Passo Fundo', 'Caxias do Sul', 'Santa Cruz do Sul - Lajeado'])]

In [88]:
#Because of Interface error between IBGE database and S2ID database, one municipality's name does not match correctly and therefore
#needs to be manually corrected
disasters_data['Município'] = disasters_data['Município'].replace(['Sant`Ana do Livramento'],  "Sant'Ana do Livramento")

In [89]:
disasters_data_by_inter_region = pd.merge(left=disasters_data, right=REGIONS_DELIMITATION, left_on='Município', right_on='nome_mun', how='left')

In [105]:
disasters_data_by_inter_region.groupby(['nome_rgint', 'Ano'])[['DH_Mortos', 'DH_Feridos', 'DH_Enfermos','DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos', 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas', 'DM_Unidades Habitacionais Destruídas', 'DM_Unidades Habitacionais Valor']].sum().groupby(['nome_rgint']).mean()

,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor
nome_rgint,,,,,,,,,,
Caxias do Sul,0.833333,13.083333,0.250000,142.250000,812.750000,0.083333,39843.750000,591.000000,16.833333,1.464231e+07
Ijuí,3005.666667,4.166667,0.250000,23.500000,97.333333,0.000000,129769.333333,1329.500000,6.916667,2.280488e+06
Passo Fundo,0.333333,8.250000,5.583333,510.000000,567.750000,0.000000,206003.916667,3046.416667,21.916667,5.765360e+06
Pelotas,0.083333,0.000000,0.083333,181.916667,622.333333,0.000000,170742.500000,2895.000000,3.250000,3.208518e+06
Porto Alegre,279.083333,873.250000,103.416667,4053.166667,32391.750000,0.333333,143168.166667,12875.583333,471.250000,3.786768e+08
Santa Cruz do Sul - Lajeado,1.416667,23.666667,2.500000,789.250000,6194.666667,91.750000,109715.833333,4701.916667,35.666667,7.003394e+07
Santa Maria,0.166667,4.166667,3.583333,178.583333,1486.750000,0.083333,172196.500000,2016.250000,7.250000,3.925024e+06
Uruguaiana,0.000000,4.750000,1.000000,361.250000,2643.833333,3.750000,87655.166667,979.416667,23.916667,3.394679e+06


## 3. Clustering of Natural Disaster Season (Year)

In [39]:
disasters_data = disasters_data[['Ano', 'COBRADE', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos',
                                 'DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos',
                                 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas',
                                 'DM_Unidades Habitacionais Destruídas',
                                 'DM_Unidades Habitacionais Valor',
                                 'DA_Poluição ou contaminação da água',
                                 'DA_Poluição ou contaminação do ar',
                                 'DA_Poluição ou contaminação do solo',
                                 'DA_Diminuição ou exaurimento hídrico'
                                ]]

In [43]:
disasters_data.groupby(['Ano']).sum()

,COBRADE,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor,DA_Poluição ou contaminação da água,DA_Poluição ou contaminação do ar,DA_Poluição ou contaminação do solo,DA_Diminuição ou exaurimento hídrico
Ano,,,,,,,,,,,,,,,
2013,13215 - Tempestade Local/Convectiva - Vendaval...,5,115,812,12380,46429,1,531830,24734,360,8.715458e+07,0,0,0,0
2014,12200 - Enxurradas13215 - Tempestade Local/Con...,1,71,29,2516,15521,0,264969,16362,229,3.035127e+07,DE 5% A 10%,0,0,0
2015,12100 - Inundações12100 - Inundações13214 - Te...,0,24,0,2908,23701,46,351192,39584,147,9.570269e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%MAIS DE 20%MAIS DE 20%,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%DE 5% A 10%DE 10% A 20%
2016,13215 - Tempestade Local/Convectiva - Vendaval...,2,346,8,1440,18321,916,243990,16636,223,7.186598e+08,DE 0% A 5%DE 5% A 10%DE 10% A 20%DE 10% A 20%D...,DE 0% A 5%,DE 0% A 5%DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 1...,DE 0% A 5%DE 10% A 20%DE 0% A 5%
2017,13213 - Tempestade Local/Convectiva - Granizo1...,5,133,1,638,18171,0,907599,30764,442,7.680782e+07,MAIS DE 20%DE 10% A 20%DE 0% A 5%DE 0% A 5%MAI...,DE 0% A 5%DE 10% A 20%,DE 0% A 5%MAIS DE 20%DE 0% A 5%MAIS DE 20%DE 0...,MAIS DE 20%DE 0% A 5%MAIS DE 20%MAIS DE 20%DE ...
2018,13213 - Tempestade Local/Convectiva - Granizo1...,4,44,4,243,2111,0,353810,9449,72,2.521029e+07,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%MAIS ...,DE 10% A 20%DE 10% A 20%DE 10% A 20%DE 10% A 20%,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 5% A 10%MAI...,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 10% A 20%DE...
2019,14110 - Estiagem14110 - Estiagem14110 - Estiag...,1,6,1,339,3579,0,501589,15539,9,2.827599e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%DE 5% A 10%DE ...,DE 0% A 5%,DE 0% A 5%DE 5% A 10%DE 0% A 5%DE 0% A 5%DE 10...,DE 5% A 10%DE 0% A 5%MAIS DE 20%DE 5% A 10%MAI...
2020,13215 - Tempestade Local/Convectiva - Vendaval...,0,8,187,17,70,0,1250456,1187,4,1.892718e+06,MAIS DE 20%MAIS DE 20%DE 5% A 10%DE 10% A 20%D...,DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 0% A 5%DE 0...,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%DE 0%...,MAIS DE 20%MAIS DE 20%MAIS DE 20%MAIS DE 20%MA...
2021,14110 - Estiagem14110 - Estiagem14110 - Estiag...,0,27,0,12,397,0,561791,38537,28,1.563735e+08,MAIS DE 20%DE 0% A 5%DE 0% A 5%,DE 0% A 5%,DE 10% A 20%,DE 5% A 10%DE 5% A 10%MAIS DE 20%MAIS DE 20%MA...
